# Analysing and Tracking Environment topic on Reddit
Su Myat Noe Yee (s3913797)

### Loading required packages

In [8]:
import sys
import json
import string
import praw
import nltk
import codecs
import re
import datetime
import nltk
import matplotlib.pyplot as mpl
import matplotlib.pyplot as plt
import pandas as pd
from colorama import Fore, Back, Style
from gensim.corpora import Dictionary
from collections import Counter
from sklearn.model_selection import cross_validate
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
import math
import pyLDAvis
import pyLDAvis.lda_model
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

Automatically reload the client information if there are changes.

In [9]:
%load_ext autoreload
%autoreload 1
%aimport redditClient
%aimport RedditProcessing
from redditClient import redditClient

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Constructing Reddit client and subreddit name we interested

In [10]:
# Construct Reddit client and check it
client = redditClient()
print(client.user.me())

Su_05


In [11]:
# Subreddit name we interested in getting the hot submissions
sSubredditName = "environment"

# Set the maximum number of hot submissions we want to retrieve
hotLimit = 200

# Get the subreddit object
subreddit = client.subreddit(sSubredditName)

### Titles of Hot Submissions

In [12]:
title_count = 0;

# Specify the target date (August 20, 2023)
target_date = datetime.datetime(2023, 8, 25)

# Print out the titles of hot submissions up until August 20, 2023
for submission in subreddit.hot(limit=hotLimit):
    submission_date = datetime.datetime.fromtimestamp(submission.created_utc)
    if submission_date.date() <= target_date.date():
        print(submission.title)
        title_count += 1

print(title_count);

0


### Users/ Authors Participation Analysis

In [13]:
lHotNames = []
lTopNames = []

# Obtain the hot submissions for this subreddit
for submission in subreddit.hot(limit=hotLimit):
    # Check if the submission has an author
    if submission.author is not None:
        # Append the author's username to the list
        lHotNames.append(submission.author.name)

# Obtain the top submissions for this subreddit
for submission in subreddit.top(limit=hotLimit):
    if submission.author is not None:
        lTopNames.append(submission.author.name)
        
# Combine the lists together
lHotNames.extend(lTopNames)
# Count the number of times each user appears in the combined list
hNameCount = Counter(lHotNames)
print(hNameCount)

Counter({'Sorin61': 17, 'Wagamaga': 17, 'pnewell': 15, 'misana123': 13, 'chrisdh79': 11, 'YoanB': 10, 'anutensil': 9, 'DukeOfGeek': 8, 'boppinmule': 7, 'lnfinity': 6, 'Fr1sk3r': 6, 'newnemo': 5, 'justin_quinnn': 5, 'Splenda': 5, 'DoremusJessup': 5, 'tta2013': 5, 'WhoIsJolyonWest': 5, 'mvea': 5, 'StoopSign': 4, 'silence7': 4, 'WilliamBlack97AI': 4, 'madazzahatter': 4, 'Huplescat22': 3, 'maki23': 3, 'sasht': 3, 'redcondurango': 3, 'maxwellhill': 3, 'AlphaXChance': 2, 'CrJ418': 2, 'FreedomsPower': 2, 'Maxcactus': 2, 'FERNnews': 2, 'Sariel007': 2, 'esporx': 2, 'Toadfinger': 2, 'Exastiken': 2, 'washingtonpost': 2, 'Ovaz1088': 2, 'ruellph': 2, 'Digimaverick': 2, 'thinkB4WeSpeak': 2, 'GoMx808-0': 2, 'morenewsat11': 2, 'SportsBetter': 2, 'AngelaMotorman': 2, 'holyfruits': 2, 'judeharte': 2, 'Dragonlance12': 2, 'HappySilveon': 2, 'ManiaforBeatles': 2, 'AdamCannon': 2, 'proctorberlin': 2, 'r4816': 2, 'pheonix200': 2, 'BringBackParis': 2, 'readerseven': 2, 'Logibenq': 1, 'ae314': 1, 'LiveScience_

In [ ]:
# Get the top 20 authors and their submission counts
top_authors = hNameCount.most_common(20)
top_author_names = [author for author, count in top_authors]
top_author_counts = [count for author, count in top_authors]

# Create a bar chart for the top 20 authors
plt.figure(figsize=(10, 6))
plt.bar(top_author_names, top_author_counts)
plt.xticks(rotation=45, ha="right")
plt.xlabel('Authors')
plt.ylabel('Number of Submissions')
plt.title('Top 20 Authors with Highest Submission Counts')
plt.tight_layout()

# Show the chart
plt.show()

### Top Commentors in Hot Submissions

In [ ]:
lCommentAuthors = []

for submission in subreddit.hot(limit=hotLimit):
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        if comment.author is not None:
            lCommentAuthors.append(comment.author.name)
            
# Count the number of times users are authors of the comments
lCommentorCount = Counter(lCommentAuthors)
print(lCommentorCount)

In [ ]:
# Get the top 20 commentors and their submission counts
top_commentors = lCommentorCount.most_common(20)
top_commentors_names = [author for author, count in top_commentors]
top_commentors_counts = [count for author, count in top_commentors]

# Create a bar chart for the top 20 commentors
plt.figure(figsize=(10, 6))
plt.bar(top_commentors_names, top_commentors_counts)
plt.xticks(rotation=45, ha="right")
plt.xlabel('Commentors')
plt.ylabel('Number of Submissions')
plt.title('Top 20 Commentors with Highest Submission Counts')
plt.tight_layout()

# Show the chart
plt.show()

### Exporting json file

In [ ]:
import json
# Fetch submissions from the specified subreddit
submissions = []

for submission in subreddit.hot(limit=hotLimit):
        submission_data = {
            "title": submission.title,
            "author": submission.author.name if submission.author else None,
            "score": submission.score,
            "comments": [
                {
                    "author": comment.author.name if comment.author else None,
                    "created": comment.created_utc,
                    "text": comment.body,
                }
                for comment in submission.comments
            ],
            "created": submission.created_utc,
        }
        submissions.append(submission_data)

# Create a dictionary to match the provided format
data = {"submissions": submissions}

# Save the data to a JSON file
output_filename = "environment.json"
with open(output_filename, "w") as json_file:
    json.dump(data, json_file, indent=4)

## Text Preprocessing
### Tokenisation removing stopwords and stemming

In [ ]:
def processText(text, tokenizer, stemmer, stopwords):
    """
    Perform tokenisation, normalisation (lower case and stemming) and stopword and twitter keyword removal.
    @param text: reddit submission or comment text
    @param tokenizer: tokeniser used.
    @param stemmer: stemmer used.
    @param stopwords: list of stopwords used
    @returns: a list of processed tokens
    """
    # Initialize counters
    total_unfiltered_tokens = 0
    total_filtered_tokens = 0

    # Covert all to lower case
    text = text.lower()

    # Tokenisation
    lTokens = tokenizer.tokenize(text)

    # Strip whitespaces before and after
    lTokens = [token.strip() for token in lTokens]

    # Remove stopwords, digits and username and non-ASCII words
    processed_tokens = []
    for tok in lTokens:
        if tok not in stopwords and not tok.isdigit() and not tok.startswith('@') and tok.isascii():
            processed_tokens.append(tok)
            
    # Return processed tokens
    return processed_tokens

In [ ]:
# Loading json file and preparation for text pre-processing
fJsonName = 'environment.json'

# Number of most frequent terms to display
freqNum = 50

# Tweet tokeniser to use
tweetTokeniser = nltk.tokenize.TweetTokenizer()

# Preparing stopwords
# Use the punctuation symbols defined in string.punctuation
lPunct = list(string.punctuation)
# Use stopwords from nltk and a few other specific terms
lStopwords = nltk.corpus.stopwords.words('english') + lPunct + ['rt', 'via', 'u', '...', '…', '"', "'", '`']

# Porter stemmer for Stemming
tweetStemmer = nltk.stem.PorterStemmer()

# Term frequency counter
termFreqCounter = Counter()

# This will store the list of posts including title and comments(Combined processed tokens) we read from subreddit for LDA analysis
lPosts_LDA = [] 

# Open json file and process it tweet by tweet
with open(fJsonName, 'r') as f:
    dSubmissions = json.load(f)
    

### Daily Count of Posts

In [ ]:
count_submission_dates = [] # Empty list to store the submission dates
 
for submission in dSubmissions['submissions']:
    submission_date = datetime.datetime.fromtimestamp(submission['created'])
    count_submission_dates.append(submission_date)

# Create a pandas DataFrame with submission dates
df = pd.DataFrame({'date': count_submission_dates})

# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Set the 'date' column as the index
df.set_index('date', inplace=True)

# Resample the data for daily and hourly counts
daily_counts = df.resample('D').size()

# Plot the time series of daily counts
plt.figure(figsize=(10, 6))
daily_counts.plot()
plt.title('Daily Count of Posts')
plt.xlabel('Date')
plt.ylabel('Number of Posts')
plt.show()

### Analyzing Term Frequency in Reddit Submissions Titles

In [ ]:
for submission in dSubmissions['submissions']:
    submissionsTitle = submission.get('title', '') 
    # Tokenise, filter stopwords and get convert to lower case
    lTokens = processText(text=submissionsTitle, tokenizer=tweetTokeniser, stemmer=tweetStemmer, stopwords=lStopwords)
    # Update count
    termFreqCounter.update(lTokens)
    # All tokens of each text are combined for further topic modelling, LDA.
    #lPosts.append(' '.join(lTokens)) 

for term, count in termFreqCounter.most_common(freqNum):
    print(term + ': ' + str(count))

In [ ]:
# Create lists of terms and their corresponding counts for plotting
top_terms = [term for term, count in termFreqCounter.most_common(freqNum)]
term_counts = [count for term, count in termFreqCounter.most_common(freqNum)]

# Create a bar chart using matplotlib
plt.figure(figsize=(10, 6))
plt.bar(top_terms, term_counts)
plt.xticks(rotation=45, ha="right")
plt.xlabel('Terms')
plt.ylabel('Term Frequency')
plt.title('Top ' + str(freqNum) + ' Most Frequent Terms in Submission Title')
plt.tight_layout()

# Display the chart
plt.show()

### Analyzing Term Frequency in Reddit Submissions Titles and Comments

In [ ]:
# Combine submission titles and comments text
texts = [] # "texts" is the combination of submission titles and comments text
for submission in dSubmissions['submissions']:
    texts.append(submission.get('title', ''))
    for comment in submission['comments']:
        texts.append(comment['text'])

# Tokenise, filter stopwords and get convert to lower case
tokenized_texts = []
for text in texts:
    tokens = processText(text, tweetTokeniser, tweetStemmer, lStopwords)
    tokenized_texts.append(tokens)

for tokens in tokenized_texts:
     # All tokens of each text are combined for further topic modelling, LDA.
    lPosts_LDA.append(' '.join(tokens)) 
    termFreqCounter.update(tokens)

for term, count in termFreqCounter.most_common(freqNum):
    print(term + ': ' + str(count))

In [ ]:
# Create lists of terms and their corresponding counts for plotting
top_terms = [term for term, count in termFreqCounter.most_common(freqNum)]
term_counts = [count for term, count in termFreqCounter.most_common(freqNum)]

# Create a bar chart using matplotlib
plt.figure(figsize=(10, 6))
plt.bar(top_terms, term_counts)
plt.xticks(rotation=45, ha="right")
plt.xlabel('Terms')
plt.ylabel('Term Frequency')
plt.title('Top ' + str(freqNum) + ' Most Frequent Terms in Submission Titles and Comments')
plt.tight_layout()

# Display the chart
plt.show()

### Token counts before and after pre-processing and cleaning submission title and comments

In [ ]:
# Initialize counters
total_original_tokens = 0
total_processed_tokens = 0

# Iterate through submissions and comments
for submission in dSubmissions['submissions']:
    # Count tokens in the original title
    original_title = submission.get('title', '')
    original_title_tokens = tweetTokeniser.tokenize(original_title)
    total_original_tokens += len(original_title_tokens)
    
    # Process title and count processed tokens
    processed_title_tokens = processText(original_title, tweetTokeniser, tweetStemmer, lStopwords)
    total_processed_tokens += len(processed_title_tokens)
    
    # Process and count tokens in comments
    for comment in submission['comments']:
        comment_text = comment['text']
        comment_tokens = tweetTokeniser.tokenize(comment_text)
        total_original_tokens += len(comment_tokens)
        
        processed_comment_tokens = processText(comment_text, tweetTokeniser, tweetStemmer, lStopwords)
        total_processed_tokens += len(processed_comment_tokens)

print("Total Original Tokens:", total_original_tokens)
print("Total Processed Tokens:", total_processed_tokens)

### Tokens Before and After Text Pre-processing

In [ ]:
# Process the first submission

first_submission = dSubmissions['submissions'][100]  # Get the first submission
first_submissionsTitle = first_submission.get('title', '')
first_lTokens = processText(text=first_submissionsTitle, tokenizer=tweetTokeniser, stemmer=tweetStemmer, stopwords=lStopwords)

# Print original and processed text only for the first submission
print("Original Text:", first_submissionsTitle)
print("Processed Text:", first_lTokens)

## Sentiment Analysis of Reddit Posts and Comments
### Approach 1: Use counts of positive and negative words to calculate the sentiment.

In [ ]:
def computeSentiment(lTokens, setPosWords, setNegWords):
    """
    Compute the overall sentiment of the list of tokens in lTokens, using the countWordSentimentAnalysis approach.
    
    @param lTokens: List of tokens to calculate the overall sentiment for.
    @param setPosWords: Set of positive words.
    @param setNegWords: Set of negative words.
    
    @returns Sentiment score for lTokens.
    """
    # count the number of positive words 
    posNum = len([tok for tok in lTokens if tok in setPosWords])
    # count the number of negative words 
    negNum = len([tok for tok in lTokens if tok in setNegWords])
    sentiment = posNum - negNum

    return sentiment

In [ ]:
def printColouredTokens(lTokens, setPosWords, setNegWords, sentiment):
    """
    If sentiment is positive, token in red
    If sentiment is negative, toekn in blue.
    Otherwise no colouring.
    
    @param lTokens: List of tokens to print and colour.
    @param setPosWords: Set of positive words.
    @param setNegWords: Set of negative words.
    @param sentiment: Sentiment score of list of tokens.
    """
    
    for token in lTokens:
        if token in setPosWords:
            print(Fore.RED + token + ', ', end='')
        elif token in setNegWords:
            print(Fore.BLUE + token + ', ', end='')
        else:
            print(Style.RESET_ALL + token + ', ', end='')

    print(': {}'.format(sentiment))

In [ ]:
def countWordSentimentAnalysis(setPosWords, setNegWords, sTweetsFilename, bPrint, redditProcessor):
    """
    Basic sentiment analysis.  Count the number of positive words, count the negative words, overall polarity is the
    difference in the two numbers.

    @param setPosWords: set of positive sentiment words
    @param setNegWords: set of negative sentiment words
    @param sFilename: name of input file containing a json formated dump
    @param bPrint: whether to print the stream of tokens and sentiment.  Uses colorama to highlight sentiment words.
    @param redditProcessor: RedditProcessing object, used to pre-process each tweet.

    @returns: list of reddit posts, in the format of [date, sentiment]
    """
    positive_count = 0
    negative_count = 0
    neutral_count = 0
    
    lSentiment = []
    # Open file and process reddit submissions and comments, one by one
    with open(sTweetsFilename, 'r') as f:
        redditDump = json.load(f)
        
        for submission in redditDump['submissions']:
            postText = submission['title']
            postDate = submission['created']
            
            # Pre-process the reddit post text
            lTokens = redditProcessor.process(postText)
            
            # Compute sentiment
            sentiment = computeSentiment(lTokens, setPosWords, setNegWords)
            
            if sentiment > 0:
                positive_count += 1
            elif sentiment < 0:
                negative_count += 1
            else:
                neutral_count += 1

                
            # Save date and sentiment of each reddit post (used for time series)
            lSentiment.append([pd.to_datetime(postDate, unit='s'), sentiment])

            # If print, each token is printed and coloured red if positive word, and blue if negative
            if bPrint:
                printColouredTokens(lTokens, setPosWords, setNegWords, sentiment)
                
            # Process the comments
            for comment in submission['comments']:
                postText = comment['text']
                postDate = comment['created']
                
                # Pre-process the text
                lTokens = redditProcessor.process(postText)
            
                # Compute sentiment
                sentiment = computeSentiment(lTokens, setPosWords, setNegWords)
                
                if sentiment > 0:
                    positive_count += 1
                elif sentiment < 0:
                    negative_count += 1
                else:
                    neutral_count += 1
                    
                # Save the date and sentiment of each tweet (used for time series)
                lSentiment.append([pd.to_datetime(postDate,unit='s'), sentiment])

                # If print, each token is printed and coloured according to positive or negative sentiment
                if bPrint:
                    printColouredTokens(lTokens, setPosWords, setNegWords, sentiment)
                    
    print("Count Method")
    print("Positive Sentiments:", positive_count)
    print("Negative Sentiments:", negative_count)
    print("Neutral Sentiments:", neutral_count)
    return lSentiment

### Approach 2: Vader based approach to sentiment analysis

In [ ]:
def vaderPrintTokens(lTokens, dSentimentScores):
    """
    Print out the tokens and sentiment score.
    
    @param lTokens: List of tokens to print and colour.
    @dSentimentScores: Dictionary of sentiment from Vader.

    """
    print(*lTokens, sep=', ')
    for cat,score in dSentimentScores.items():
        print('{0}: {1}, '.format(cat, score), end='')
    print()

In [ ]:
def vaderSentimentAnalysis(sTweetsFilename, bPrint, tweetProcessor):
    """
    Use Vader lexicons instead of a raw positive and negative word count.

    @param sTweetsFilename: name of input file containing a json formated tweet dump
    @param bPrint: whether to print the stream of tokens and sentiment.  Uses colorama to highlight sentiment words.
    @param tweetProcessor: TweetProcessing object, used to pre-process each tweet.

    @returns: list of tweets, in the format of [date, sentiment]
    """

    # Setting up vader sentiment analyser, part of nltk
    sentAnalyser = SentimentIntensityAnalyzer()
    
    # Counting sentiments for Vader analysis
    positive_count = 0
    negative_count = 0
    neutral_count = 0

    lSentiment = []
    # Open file and process tweets, one by one
    with open(sTweetsFilename, 'r') as f:
        redditDump = json.load(f)
        
        for submission in redditDump['submissions']:
            postText = submission['title']
            postDate = submission['created']
            
            #Pre-process the post text
            lTokens = redditProcessor.process(postText)

            #Computes sentiment scores(polarity score in nltk)
            dSentimentScores = sentAnalyser.polarity_scores(" ".join(lTokens))
            
            sentiment = dSentimentScores['compound']
            if sentiment > 0.05:  #Positive sentiment threshold
                positive_count += 1
            elif sentiment < -0.05:  #Negative sentiment threshold
                negative_count += 1
            else:
                neutral_count += 1
                
            #Save date and sentiment of each post (used for time series)
            lSentiment.append([pd.to_datetime(postDate, unit='s'), dSentimentScores['compound']])

            #If print, print tokens then the sentiment scores
            #Cannot use colorama to label each token as there's  no list of positive and negative words
            if bPrint:
                vaderPrintTokens(lTokens, dSentimentScores)
                
            #Process the comments
            for comment in submission['comments']:
                postText = comment['text']
                postDate = comment['created']
                
                #Pre-process the post text
                lTokens = redditProcessor.process(postText)

                #Computes sentiment scores (polarity score in nltk)
                dSentimentScores = sentAnalyser.polarity_scores(" ".join(lTokens))

                #Save the date and sentiment of each post (used for time series)
                lSentiment.append([pd.to_datetime(postDate, unit='s'), dSentimentScores['compound']])
                
                sentiment = dSentimentScores['compound']
                if sentiment > 0.05:
                    positive_count += 1
                elif sentiment < -0.05:
                    negative_count += 1
                else:
                    neutral_count += 1

                #If print, print tokens then the sentiment scores. 
                #Cannot use colorama to label each token as there's  no list of positive and negative words
                if bPrint:
                    vaderPrintTokens(lTokens, dSentimentScores)
                    
    print("Vader Method")
    print("Positive Sentiments:", positive_count)
    print("Negative Sentiments:", negative_count)
    print("Neutral Sentiments:", neutral_count)

    return lSentiment

In [ ]:
#Input set of postive words file 
posWordFile = 'positive-words.txt'
#Input set of negative words file
negWordFile = 'negative-words.txt'
#Input set of reddit posts files in json format
redditFile = 'environment.json'
#Flag to determine whether to print out tweets and their sentiment
flagPrint = True
#Specify the approach to take (count or vader)
approach = 'count'

In [ ]:
#Call the RedditProcessing python script
redditProcessor = RedditProcessing.RedditProcessing(tweetTokeniser, lStopwords)

# Load set of positive words
lPosWords = []
with open(posWordFile, 'r', encoding='utf-8', errors='ignore') as fPos:
    for sLine in fPos:
        lPosWords.append(sLine.strip())
setPosWords = set(lPosWords)

# load set of negative words
lNegWords = []
with codecs.open(negWordFile, 'r', encoding='utf-8', errors='ignore') as fNeg:
    for sLine in fNeg:
        lNegWords.append(sLine.strip())
setNegWords = set(lNegWords)

In [ ]:
# Compute the sentiment
lSentiment = []
if approach == 'count':
    lSentiment = countWordSentimentAnalysis(setPosWords, setNegWords, redditFile, flagPrint, redditProcessor)
elif approach == 'vader':
    lSentiment = vaderSentimentAnalysis(redditFile, flagPrint, redditProcessor)

In [ ]:
# Apply the "Vader Method" and "Count Method" separately
vader_sentiment = vaderSentimentAnalysis(redditFile, flagPrint, redditProcessor)
count_sentiment = countWordSentimentAnalysis(setPosWords, setNegWords, redditFile, flagPrint, redditProcessor)

# Create DataFrames for each sentiment analysis method
vader_series = pd.DataFrame(vader_sentiment, columns=['date', 'vader_sentiment'])
vader_series.set_index('date', inplace=True)

count_series = pd.DataFrame(count_sentiment, columns=['date', 'count_sentiment'])
count_series.set_index('date', inplace=True)

# Resample sentiment scores on an hourly basis
vader_resampled = vader_series.resample('1H').sum()
count_resampled = count_series.resample('1H').sum()

# Combine the two DataFrames into a single DataFrame
combined_series = pd.concat([vader_resampled, count_resampled], axis=1)

# Plot the combined DataFrame
combined_series.plot()
plt.show()

### Text Analysis and Topic Modeling of Reddit Submissions and Comments

In [ ]:
# Number of topics to discover
topicNum = 5
# Maximum number of words to display per topic
wordNumToDisplay = 15
# Number of features/words to used to describe our documents
featureNum = 2000

In [ ]:
tfVectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=featureNum, stop_words='english')
tf = tfVectorizer.fit_transform(lPosts_LDA)
# Extract the names of the features 
tfFeatureNames = tfVectorizer.get_feature_names_out()

# Run LDA (see documentation about what the arguments means)
ldaModel = LatentDirichletAllocation(n_components =topicNum, max_iter=10, learning_method='online').fit(tf)

In [ ]:
def display_topics(model, featureNames, numTopWords):
    """
    Prints out the most associated words for each feature.

    @param model: lda model.
    @param featureNames: list of strings, representing the list of features/words.
    @param numTopWords: number of words to print per topic.
    """
    # Print out the topic distributions
    for topicId, lTopicDist in enumerate(model.components_):
        print("Topic %d:" % (topicId+1))
        print(" ".join([featureNames[i] for i in lTopicDist.argsort()[:-numTopWords - 1:-1]]))

In [ ]:
display_topics(ldaModel, tfFeatureNames, wordNumToDisplay)

In [ ]:
panel = pyLDAvis.lda_model.prepare(ldaModel, tf, tfVectorizer, mds='tsne')
pyLDAvis.display(panel)

In [ ]:
def displayWordcloud(model, featureNames):
    """
    Displays the word cloud of the topic distributions, stored in model.

    @param model: lda model.
    @param featureNames: list of strings, representing the list of features/words.
    """

    # This normalises each row/topic to sum to one, use this normalisedComponents to display your wordclouds
    normalisedComponents = model.components_ / model.components_.sum(axis=1)[:, np.newaxis]

    topicNum = len(model.components_)
    # Number of wordclouds for each row
    plotColNum = 3
    # Number of wordclouds for each column
    plotRowNum = int(math.ceil(topicNum / plotColNum))

    for topicId, lTopicDist in enumerate(normalisedComponents):
        lWordProb = {featureNames[i] : wordProb for i,wordProb in enumerate(lTopicDist)}
        wordcloud = WordCloud(background_color='black')
        wordcloud.fit_words(frequencies=lWordProb)
        plt.subplot(plotRowNum, plotColNum, topicId+1)
        plt.title('Topic %d:' % (topicId+1))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")

    plt.show(block=True)

In [ ]:
# display wordcloud
displayWordcloud(ldaModel, tfFeatureNames)